In [82]:
import openai
import pandas as pd
import os
import json
import numpy as np

전체 출력 설정
pd.set_option('display.max_colwidth', None)  # 한 셀에 출력될 최대 너비를 제한 없이 설정

In [109]:
data=pd.read_csv("오픈마켓 노쇼핑 샘플 데이터.csv")
# OpenAI API Key를 환경 변수에서 가져오기
openai.api_key = os.getenv("OPENAI_API_KEY")

# 통합 데이터 X 각자 데이터 샘플링해서 시도한 코드
data = pd.read_csv("C:/Users/sera/Desktop/WECOMMIT_노쇼핑/번개장터sampledata_prompt테스트용.csv")
df_hotel = data[data['category'].str.contains('여행', na=False)]
df_hotel_sample = df_hotel.sample(n=20, random_state=42)
df_hotel_sample
# 데이터 프레임의 각 행에 대해 필드 추출 적용
df_hotel_sample['field'] = df_hotel_sample['description'].apply(extract_fields)

# 결과 확인
print(df_hotel_sample[['description', 'field']])

df_hotel_sample = df_hotel.sample(n=5, random_state=42)
df_hotel_sample
# 데이터 프레임의 각 행에 대해 필드 추출 적용
df_hotel_sample['field'] = df_hotel_sample['description'].apply(extract_fields)

# 결과 확인
print(df_hotel_sample[['description', 'field']])

jung_df=pd.read_csv("C:/Users/sera/Desktop/WECOMMIT_노쇼핑/중고나라_sample_data.csv")
jung_df.head()

jung_df_sample = jung_df.sample(n=5)
jung_df_sample
# 데이터 프레임의 각 행에 대해 필드 추출 적용
jung_df_sample['field'] = jung_df_sample['C10'].apply(extract_fields)

# 결과 확인
print(jung_df_sample[['C10', 'field']])

In [110]:
def extract_fields(description):
    prompt = f"""
    우리는 여러 플랫폼에서 크롤링한 데이터를 통해 호텔과 숙박에 관한 노쇼 상품들을 판매할거야. 
    기본 필드에서 가져올 수 있는 정보들을 제외하고 우리가 필요한 필드가 10개가 있어. 
    만약 NaN이라면 그냥 모든 필드를 JSON의 `null`로 표시해줘.
    이 9개의 필드는 
    1. 노쇼 상품의 사용 날짜 및 기한을 나타내는 expiration_date
       이때 날짜가 범위로 표현되어 있다면 범위를 나타내고, 날짜가 없다면 JSON의 `null`로 표기해줘. 날짜 형식은 20xx-mm-dd로 통일해줘
    2. 객실 타입을 나타내는 room_type
       문자열 형태이며 관련된 내용이 없다면 JSON의 `null`로 입력해줘.
    3. 사용 인원 수를 나타내는 head_count
       데이터 타입은 int type이고 성인이나 자녀에 대한 구분 없이 총 인원수만 입력해줘. 인원수에 대한 내용이 없다면 JSON의 `null`로 기입해줘.
    4. 배송비인 shipping_fee 
       데이터 타입은 int type이고, 배송비가 적혀있지 않다면 0으로 표기해줘 (숫자 형식).
    5. 직거래인지 택배거래인지 알 수 있는 거래 유형인 transaction_method
       직거래라는 단어가 있으면 '직거래'로, 택배거래라는 단어가 있으면 '택배거래'로 표기해줘.
       두 단어가 모두 없으면 JSON의 `null`로 표기해줘.
    6. 원래 노쇼 상품의 가격인 시중 가격인 market_price 
       데이터 타입은 int type이고, 시중 가격이 나와 있지 않다면 JSON의 `null`로 표기해줘
       만약 23.3만원 이런식으로 적혀져 있다면 market_price는 233000으로 입력되어야해.
    7. 조식 여부나 사우나 여부, 수영장 여부 등의 호텔에서 사용할 수 있는 서비스 이용 여부를 알 수 있는 옵션인 options
       서비스 관련 내용이 없다면 JSON의 `null`로 표기해줘.
    8. 주차 여부를 나타내는 parking
       주차 여부가 있으면 TRUE로, 없으면 FALSE로 표기해줘. 주차 여부 정보가 없다면 JSON의 `null`로 표기해줘.
    9. 호텔 체크인 시간을 나타내는 check_in_time
       체크인 시간을 time 형태로 작성해줘. 
       만약 체크인 시간이 없으면 JSON의 `null`로 작성해줘.
    10. 호텔 체크아웃 시간을 나타내는 check_out_time
       체크아웃 시간을 time 형태로 작성해줘. 
       만약 체크아웃 시간이 없으면 JSON의 `null`로 작성해줘.
    설명에서 필요한 10개의 필드 정보만 추출해 json 형태로 작성해줘:
    JSON 형식으로 작성해줘. 예시: 

        "expiration_date": "2023-10-01",
        "room_type": "더블룸",
        "head_count": 2,
        ...
    
    설명: {description}

    필드:
    - expiration_date:
    - room_type:
    - head_count:
    - shipping_fee:
    - transaction_method:
    - market_price:
    - options:
    - parking:
    - check_in_time:
    - check_out_time:
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=1000
    )
    

    result = response['choices'][0]['message']['content'].strip()
    
    print("API 반환 결과:", result)
    
# json으로 시작되는 쓸데없는 부분을 제거 / 끝부분에 '들어가는 부분 제거 코드
    if result.startswith("```json"):
        result = result.replace("```json", "").strip()
    if result.endswith("```"):
        result = result.replace("```", "").strip()
        
   # 'null'을 실제 JSON의 null로 변환
    result = result.replace('"null"', 'null')

    try:
        return json.loads(result)
    except json.JSONDecodeError:
        print("유효하지 않은 JSON 형식입니다.")
        return None

In [76]:
# 1행은 그냥 빈 칸이라 뺌.
df = data.iloc[1:,:]

In [106]:
df_sample = df.sample(n=5)
json_to_df= df_sample['description'].apply(extract_fields)
# 데이터프레임으로 만드는 코드
fields_df = pd.json_normalize(json_to_df)
fields_df

API 반환 결과: {
    "expiration_date": null,
    "room_type": null,
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "직거래",
    "market_price": 6000,
    "options": null,
    "parking": null,
    "check_in_out_time": null
}
API 반환 결과: ```json
{
    "expiration_date": "2022-08-16 / 2023-02-16",
    "room_type": null,
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": "직거래",
    "market_price": 300000,
    "options": null,
    "parking": null,
    "check_in_out_time": null / null
}
```
유효하지 않은 JSON 형식입니다.
API 반환 결과: {
    "expiration_date": "2024-12-18",
    "room_type": null,
    "head_count": 2,
    "shipping_fee": 3500,
    "transaction_method": "직거래",
    "market_price": null,
    "options": null,
    "parking": null,
    "check_in_out_time": "오전 / 오후 6시 30분"
}
API 반환 결과: ```json
{
    "expiration_date": null,
    "room_type": null,
    "head_count": null,
    "shipping_fee": 0,
    "transaction_method": null,
    "market_price": null,
    "

,expiration_date,room_type,head_count,shipping_fee,transaction_method,market_price,options,parking,check_in_out_time
0,None,None,NaN,0.0,직거래,6000.0,NaN,NaN,None
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-18,None,2.0,3500.0,직거래,NaN,NaN,NaN,오전 / 오후 6시 30분
3,None,None,NaN,0.0,None,NaN,NaN,NaN,None
4,None,Deluxe Premier Twin,NaN,0.0,None,NaN,NaN,NaN,Check-in time: null / Check-out time: null


In [107]:
fields_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   expiration_date     1 non-null      object 
 1   room_type           1 non-null      object 
 2   head_count          1 non-null      float64
 3   shipping_fee        4 non-null      float64
 4   transaction_method  2 non-null      object 
 5   market_price        1 non-null      float64
 6   options             0 non-null      float64
 7   parking             0 non-null      float64
 8   check_in_out_time   2 non-null      object 
dtypes: float64(5), object(4)
memory usage: 488.0+ bytes
